<a href="https://colab.research.google.com/github/ansharyis/ml-colab-project/blob/main/notebooks/01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_DIR = "/content/drive/MyDrive/ML_Project_Data"


In [ ]:
# Project: ML Weight Prediction
# Notebook: Preprocessing Data
# Owner:
# Description: This Notebook is focus to perform pre-processing the data before analysis is taken place.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

train_df = pd.read_excel(f"{DATA_DIR}/RAW/training_dataset.xlsx")
vars_df  = pd.read_excel(f"{DATA_DIR}/RAW/nhis variable names 2020.xlsx")

print(train_df.shape)
print(vars_df.shape)


(20340, 617)
(616, 1)


## STEP 01: PREPROCESSING DATA

Before performing any analytics into the data, we will performs some preprocessing steps to ensure the data used is clean enough:  
1.   Detecting outlier: impossible values
2.   Missing values: Survey-coded missing (e.g. 7, 9, 97, 99, 7777)
1.   Constant values: Columns with zero or near-zero variance
2.   Null Value, NaN, and empty


### 1. Detecting Outlier: impossible values

To detect outlier we will construct confidence interval for each feature using confidence level 95% (Z crit = 1.96). In order to build more robust model, we will provide every feature with treshold that constructed using

$$upperbound = μ + 1.96 \sigma$$
$$lowerbound = μ - 1.96 \sigma$$

where:

$μ$: Median of the feature

$σ$: 1.48MAD

In [ ]:
Z = 1.96
MAD_SCALE = 1.4826  # robust std approx

def robust_bounds(series: pd.Series, z: float = Z, mad_scale: float = MAD_SCALE):
    """
    Compute robust lower/upper bounds using:
      mu = median
      sigma = 1.4826 * MAD
      bounds = mu +/- z * sigma
    """
    x = pd.to_numeric(series, errors="coerce")  # non-numeric -> NaN
    mu = np.nanmedian(x)

    # MAD = median(|x - median(x)|)
    mad = np.nanmedian(np.abs(x - mu))
    sigma = mad_scale * mad

    # If sigma is 0 (constant / near-constant), bounds collapse to mu
    lower = mu - z * sigma
    upper = mu + z * sigma

    return mu, mad, sigma, lower, upper

def detect_outliers_robust_ci(
    train_df: pd.DataFrame,
    exclude_cols=None,
    z: float = Z,
    mad_scale: float = MAD_SCALE,
    numeric_only: bool = True
):
    """
    Returns:
      bounds_df: per-column mu, MAD, sigma, lower, upper, outlier_count, outlier_rate
      outlier_mask_df: boolean df (True if outlier) for each processed column
    """
    if exclude_cols is None:
        exclude_cols = []

    df = train_df.copy()

    # Choose columns
    if numeric_only:
        cols = df.select_dtypes(include=[np.number]).columns.tolist()
    else:
        cols = df.columns.tolist()

    cols = [c for c in cols if c not in exclude_cols]

    bounds_records = []
    outlier_mask_df = pd.DataFrame(index=df.index)

    for col in cols:
        x = pd.to_numeric(df[col], errors="coerce")

        mu, mad, sigma, lower, upper = robust_bounds(x, z=z, mad_scale=mad_scale)

        mask = (x < lower) | (x > upper)
        # keep NaNs as False (missing is not "outlier" here)
        mask = mask.fillna(False)

        outlier_mask_df[col] = mask

        outlier_count = int(mask.sum())
        non_missing = int(x.notna().sum())

        bounds_records.append({
            "feature": col,
            "median_mu": mu,
            "mad": mad,
            "sigma_1.48mad": sigma,
            "lowerbound": lower,
            "upperbound": upper,
            "non_missing_n": non_missing,
            "outlier_count": outlier_count

        })

    bounds_df = pd.DataFrame(bounds_records).sort_values("outlier_count", ascending=False)
    return bounds_df, outlier_mask_df


# Example: exclude target from being treated as a feature
TARGET = "WEIGHTLBTC_A"

bounds_df, outlier_mask_df = detect_outliers_robust_ci(
    train_df,
    exclude_cols=[TARGET],
    numeric_only=True
)

bounds_df.head(100)


/tmp/ipython-input-3611167523.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  outlier_mask_df[col] = mask
/tmp/ipython-input-3611167523.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  outlier_mask_df[col] = mask
/tmp/ipython-input-3611167523.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()

,feature,median_mu,mad,sigma_1.48mad,lowerbound,upperbound,non_missing_n,outlier_count
124,PCNT18UPTC,2.0,0.0,0.0,2.0,2.0,20340,10150
302,WEATHERWLK_A,3.0,0.0,0.0,3.0,3.0,20340,10073
408,EYEEX12M_A,1.0,0.0,0.0,1.0,1.0,20340,10014
308,FUNWLK_A,2.0,0.0,0.0,2.0,2.0,20340,9803
110,MODTPR_A,2.0,0.0,0.0,2.0,2.0,20340,9696
...,...,...,...,...,...,...,...,...
266,EMPPDSKLV_A,1.0,0.0,0.0,1.0,1.0,11225,3548
522,SINCOVRX_A,2.0,0.0,0.0,2.0,2.0,20340,3540
422,SHTFLUY_A,2019.0,0.0,0.0,2019.0,2019.0,11026,3527
570,DEPEV_A,2.0,0.0,0.0,2.0,2.0,20340,3527


In [ ]:
def replace_outliers_with_nan(
    train_df: pd.DataFrame,
    bounds_df: pd.DataFrame,
    exclude_cols=None
):
    """
    Replace outliers (outside robust CI bounds) with NaN.

    bounds_df must contain:
    ['feature', 'lowerbound', 'upperbound']
    """
    if exclude_cols is None:
        exclude_cols = []

    df = train_df.copy()
    bounds_map = bounds_df.set_index("feature")[["lowerbound", "upperbound"]]

    for col in bounds_map.index:
        if col in df.columns and col not in exclude_cols:
            x = pd.to_numeric(df[col], errors="coerce")
            lower = bounds_map.loc[col, "lowerbound"]
            upper = bounds_map.loc[col, "upperbound"]

            outlier_mask = (x < lower) | (x > upper)
            df.loc[outlier_mask, col] = np.nan

    return df


After detecting outlier value, we will replace the outlier vlaue with Null value, meaning that we want to eliminate that outlier into the model to ensure that model run only on correct data within confidence level 95%.

In [ ]:
TARGET = "WEIGHTLBTC_A"

train_df_outlier_nan = replace_outliers_with_nan(
    train_df=train_df,
    bounds_df=bounds_df,
    exclude_cols=[TARGET]
)


In [ ]:
nan_before = train_df.isna().sum().sum()
nan_after = train_df_outlier_nan.isna().sum().sum()

print("NaN before outlier handling:", nan_before)
print("NaN after outlier handling :", nan_after)
print("New NaNs added by outlier handling:", nan_after - nan_before)


NaN before outlier handling: 6449806
NaN after outlier handling : 7453157
New NaNs added by outlier handling: 1003351


### 2. Missing values: Survey-coded missing (e.g. 7, 9, 97, 99, 7777)

This section will focus on replacing survey-coded missing value and replace with the expectation for each feature. We choose to replace the category missing value with Null value to make it not considered in the model. Based on this NHIS 2020 Sample Adult Codebook, we categorized several missing data as below:



> Survey-coded missing
1. Universal Non-Substantive Codes (Most Variables). These appear across almost all sections (ex: MEDICARE_A, SINCOVRX_A, RECTUCAN_A, INJBONES_A, CIGNOW_A, etc.)

$$
\begin{array}{lcc}
\hline
\text{Code} & \text{Meaning} & Notes \\
\hline
7 & Refused Respondent & Explicitly Refused \\
8 & Not Ascertained & Question Could Not Be Determined \\
9 & Don’t Know & Respondent Did Not Know \\
\hline
\end{array}
$$


2. Extended Numeric Missing Codes (Numeric Variables). Used when variables have ranges (cost, age, counts, duration).
$$
\begin{array}{lcc}
\hline
\text{Code} & \text{Meaning} & Notes \\
\hline
97 & Refused & Age, workdays, cigarettes \\
98 & Not Ascertained & Activity, smoking, injury \\
99 & Don’t Know & Age, counts, duration \\
\hline
\end{array}
$$

3. Cost & Monetary Variables (High-Value Codes) for example HICOSTR1_A (annual insurance premium)
$$
\begin{array}{lcc}
\hline
\text{Code} & \text{Meaning}\\
\hline
99997 & Refused  \\
99998 & Not  Ascertained  \\
99999 & Don’t Know \\
\hline
\end{array}
$$

4. “Not Applicable / Not in Universe. These must NOT be treated as missing randomly. fpr example HHSTAT_A ≠ 1 → many adult questions not asked
$$
\begin{array}{lcc}
\hline
\text{Code} & \text{Pattern} & Meaning \\
\hline
Blank & \text{Not In Universe} & Not In Universe \\
0 & \text{Legitimate} & structural zero \\
Universe & Restriction & Question not asked \\
\hline
\end{array}
$$

5. Extreme / Confidentiality Codes (SHOULD BE SET TO NA). These are not real values. For example: VIGFREQW_A, VIGLNR_A

$$
\begin{array}{lcc}
\hline
\text{Code} & \text{Meaning} & Variable \\
\hline
95 / 96 & Extreme / Unable & Physical activity \\
996 & Not available (confidentiality)& Weight \\
9995 / 9996	 & Extreme value & Activity duration \\
\hline
\end{array}
$$

6. Categorical “Unknown” Codes. for example BMICAT_A
$$
\begin{array}{lcc}
\hline
\text{Code} & \text{Meaning} \\
\hline
9 & Unkown \\
\hline
\end{array}
$$

In [ ]:
import numpy as np
import pandas as pd

# ==============================
# NHIS Survey-Coded Missing
# ==============================

NHIS_MISSING_CODES = {
    "universal": {7, 8, 9},
    "numeric": {97, 98, 99},
    "cost": {99997, 99998, 99999},
    "extreme": {95, 96, 996, 9995, 9996},
    "occupation": {9997, 9998, 9999}
}

# Flatten all survey-coded missing values
ALL_SURVEY_MISSING = set().union(*NHIS_MISSING_CODES.values())

def detect_survey_missing_per_column(df: pd.DataFrame) -> dict:
    """
    Detect which NHIS survey-coded missing values
    appear in each column.
    """
    survey_missing_map = {}

    for col in df.columns:
        if not pd.api.types.is_numeric_dtype(df[col]):
            continue

        present_codes = set(df[col].dropna().unique())
        detected = present_codes.intersection(ALL_SURVEY_MISSING)

        if detected:
            survey_missing_map[col] = sorted(detected)

    return survey_missing_map

survey_missing_by_column = detect_survey_missing_per_column(train_df_outlier_nan)
survey_missing_by_column


{'RATCAT_A': [np.float64(7.0), np.float64(8.0), np.float64(9.0)],
 'PPSU': [7, 8, 9],
 'WLKLEISTC_A': [7, 8, 9],
 'WLKTRANTC_A': [7, 8, 9],
 'MARSTAT_A': [np.float64(7.0), np.float64(8.0), np.float64(9.0)],
 'PRTNREDUC_A': [np.float64(7.0), np.float64(8.0), np.float64(9.0)],
 'SPOUSEDUC_A': [np.float64(7.0), np.float64(8.0), np.float64(9.0)],
 'EMDOCCUPR2_A': [7, 8, 9],
 'EMDOCCUPR1_A': [7, 8, 9, 97, 98, 99],
 'EMDINDSTR2_A': [7, 8, 9],
 'DIFYRSTC_A': [7, 8, 9],
 'SMKQTNP_A': [7, 8, 9],
 'SMKQTY_A': [7, 8, 9],
 'DRKSTAT_A': [np.float64(7.0), np.float64(8.0)],
 'DRK12MYR_A': [7, 8, 9],
 'DRK12MWK_A': [np.float64(7.0)],
 'MODFREQW_A': [np.float64(7.0), np.float64(8.0), np.float64(9.0)],
 'MODNR_A': [np.float64(7.0)],
 'MODLNR_A': [7, 8, 9],
 'MODMIN_A': [7, 8, 9],
 'VIGLNR_A': [7, 8, 9],
 'VIGMIN_A': [7, 8],
 'COVER65_A': [np.float64(7.0)],
 'HICOSTR2_A': [96],
 'HICOSTR1_A': [96, 996],
 'EDUC_A': [np.float64(7.0), np.float64(8.0), np.float64(9.0)],
 'MAXEDUC_A': [np.float64(7.0), np.flo

In [ ]:
def replace_survey_missing_with_nan(df: pd.DataFrame) -> pd.DataFrame:
    """
    Replace NHIS survey-coded missing values with NaN.
    """
    df_clean = df.copy()

    for col in df_clean.columns:
        if pd.api.types.is_numeric_dtype(df_clean[col]):
            df_clean[col] = df_clean[col].replace(ALL_SURVEY_MISSING, np.nan)

    return df_clean
train_df_clean = replace_survey_missing_with_nan(train_df_outlier_nan)


In [ ]:
def missing_value_audit(df_raw: pd.DataFrame, df_clean: pd.DataFrame) -> pd.DataFrame:
    """
    Create a missing-value audit table comparing
    raw vs cleaned data.
    """
    audit = []

    for col in df_raw.columns:
        raw_missing = df_raw[col].isna().sum()
        clean_missing = df_clean[col].isna().sum()

        survey_missing = (
            df_raw[col]
            .isin(ALL_SURVEY_MISSING)
            .sum()
            if pd.api.types.is_numeric_dtype(df_raw[col])
            else 0
        )

        audit.append({
            "feature": col,
            "raw_missing_n": raw_missing,
            "survey_missing_n": survey_missing,
            "clean_missing_n": clean_missing,
            "total_rows": len(df_raw),
            "clean_missing_ratio": clean_missing / len(df_raw)
        })

    return (
        pd.DataFrame(audit)
        .sort_values("clean_missing_ratio", ascending=False)
        .reset_index(drop=True)
    )
audit_table = missing_value_audit(train_df, train_df_clean)
audit_table.head(15)


,feature,raw_missing_n,survey_missing_n,clean_missing_n,total_rows,clean_missing_ratio
0,PRPLCOV2_C_A,20340,0,20340,20340,1.000000
1,CHFLG_A,20340,0,20340,20340,1.000000
2,OGFLG_A,20340,0,20340,20340,1.000000
3,CHHDHP_A,20338,0,20338,20340,0.999902
4,OPFLG_A,20337,0,20337,20340,0.999853
5,GALLBAGETC_A,20337,0,20337,20340,0.999853
6,LARYNAGETC_A,20337,0,20337,20340,0.999853
7,CHDEDUC_A,20332,1,20333,20340,0.999656
8,ESOPHAGETC_A,20332,1,20333,20340,0.999656
9,CHPREM_A,20332,0,20332,20340,0.999607


### 3. Constant values: Columns with zero variance

We also want to filter feature parameter that contain only constant value with zero variance becasue that feature wont give any impact to the model and act like constant.

In [ ]:

def detect_zero_variance_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Detect features with zero variance
    (only one unique non-missing value).
    """
    results = []

    for col in df.columns:
        if not pd.api.types.is_numeric_dtype(df[col]):
            continue

        unique_non_null = df[col].dropna().nunique()

        if unique_non_null == 1:
            results.append({
                "feature": col,
                "unique_non_missing_values": unique_non_null,
                "constant_value": df[col].dropna().iloc[0]
            })

    return pd.DataFrame(results)
zero_variance_audit = detect_zero_variance_features(train_df_clean)
zero_variance_audit



,feature,unique_non_missing_values,constant_value
0,INCTCFLG_A,1,0.0
1,IMPINCFLG_A,1,0.0
2,RECJOBSD_A,1,2.0
3,SDMSRS_A,1,2.0
4,CURJOBSD_A,1,1.0
...,...,...,...
452,PROXYREL_A,1,1.0
453,PROXY_A,1,1.0
454,AVAIL_A,1,1.0
455,HHSTAT_A,1,1.0


In [ ]:
def remove_zero_variance_features(
    df: pd.DataFrame,
    zero_var_df: pd.DataFrame
) -> pd.DataFrame:
    features_to_drop = zero_var_df["feature"].tolist()
    return df.drop(columns=features_to_drop)


In [ ]:
train_df_final = remove_zero_variance_features(
    train_df_clean,
    zero_variance_audit
)


### 4. Handling Null, and empty Value

This part is done by evaluating null ratio fro 616 variable given in the training dataset. The focus is to determine weather variable contain to many null value and become statistically insignificant to become feature in the model. We will eliminate feature that have Null ratio > 0.5 meaning that more than 50% of the row in that feature is Null, and/or an empty value.   

In [ ]:
null_summary = (
    train_df_final.isna()
    .mean()
    .sort_values(ascending=False)
    .to_frame("null_ratio")
)

high_null_cols = null_summary[null_summary["null_ratio"] > 0.5].index

train_df_final = train_df_final.drop(columns=high_null_cols)

print(
    f"Dropped {len(high_null_cols)} variables with null ratio > 50%"
)


Dropped 101 variables with null ratio > 50%


In [ ]:
print(f"the remaining feature parameter to be used are: {train_df_final.shape[1]}")




the remaining feature parameter to be used are: 59


In [ ]:
train_df_final.columns.tolist()

['URBRRL',
 'RATCAT_A',
 'INCGRP_A',
 'FAMINCTC_A',
 'SUPPORT_A',
 'PPSU',
 'PSTRAT',
 'LEGMSTAT_A',
 'MARSTAT_A',
 'PCNTADTWFP_A',
 'PCNTADTWKP_A',
 'EMDOCCUPR2_A',
 'EMDOCCUPR1_A',
 'EMDINDSTR2_A',
 'EMDINDSTR1_A',
 'BMICAT_A',
 'WEIGHTLBTC_A',
 'HEIGHTTC_A',
 'DRKAVG12M1_A',
 'DRK12MYR_A',
 'DRK12MWK_A',
 'DRK12MN1_A',
 'DRK12MTP1_A',
 'PA18_05R_A',
 'PA18_02R_A',
 'MODFREQW_A',
 'MODNR_A',
 'MODLNR_A',
 'MODMIN_A',
 'PCNTTC',
 'EDUC_A',
 'PCNTFAM_A',
 'PCNTADLT_A',
 'REGION',
 'INTV_QRT',
 'AGEP_A',
 'HOUYRSLIV_A',
 'MARITAL_A',
 'SLPSTY_A',
 'SLPFLL_A',
 'SLPREST_A',
 'SUNSCREEN_A',
 'SUNSHIRT_A',
 'SUNHAT_A',
 'SUNSHADE_A',
 'SUNSKIN_A',
 'TRANSITWLK_A',
 'DRKAVG12M_A',
 'DRK12MTP_A',
 'DRK12MN_A',
 'FGELEVTRD_A',
 'PAIAMNT_A',
 'PAIFRQ3M_A',
 'ANXFREQ_A',
 'PHSTAT_A',
 'INTV_MON',
 'WTFA_A',
 'HHX',
 'POVRATTC_A']

In [ ]:
# Save final dataset after null-based feature deletion
output_path = "train_df_final_after_null_removal.csv"

train_df_final.to_csv(
    output_path,
    index=False
)

print(f"Final dataset saved to: {output_path}")
print(f"Shape: {train_df_final.shape}")


Final dataset saved to: train_df_final_after_null_removal.csv
Shape: (20340, 59)


In [ ]:
from google.colab import files
files.download("train_df_final_after_null_removal.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>